<a href="https://colab.research.google.com/github/martinpius/TensorFlow_Graph_Functions/blob/main/Tensorflow_Keras_Layers_and_Models_Building_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Setting up the colab environment:
from google.colab import drive
try:
  drive.mount("/content/drive", force_remount = True)
  COLAB = True
  import tensorflow as tf
  print(f"You are using Google colab with tensorflow version: {tf.__version__}")
except:
  COLAB = False
  print('Please setup the colab environment:')

def time_fmt(x):
  h = int(x/(60*60))
  m = int(x%(60*60)/60)
  s = int(x%60)
  return f"{h}: {m:>03}: {s:>05.2f}"

Mounted at /content/drive
You are using Google colab with tensorflow version: 2.3.0


In [2]:
#Importing necessary packs
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import timeit
from datetime import datetime
plt.style.use('fivethirtyeight')
%load_ext tensorboard
%matplotlib inline

In [3]:
#Creating, saving and resoring tensorflow models from scratch (tf building blocks (layers, models, variables))

In [6]:
#A simple tensorflow module example
class MytfModule(tf.Module):
  def __init__(self, name = None):
    super(MytfModule, self).__init__(name = name)
    self.var_train = tf.Variable(initial_value = 3.1467, trainable = True, name = 'trainable_variable')
    self.var_non_train = tf.Variable(initial_value = 12.8, trainable = False, name = 'non_trainable_variable')
  
  def __call__(self, inputs):
    out = tf.multiply(self.var_train, inputs) + self.var_non_train
    return out
mymodule = MytfModule(name = 'test')
mymodule(tf.constant(2, dtype= tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=19.0934>

In [7]:
#Subclassing
print(f"Total variables: {mymodule.variables}\nTrainable variables: {mymodule.trainable_variables}\nNon trainable variables: {mymodule.var_non_train}")

Total variables: (<tf.Variable 'non_trainable_variable:0' shape=() dtype=float32, numpy=12.8>, <tf.Variable 'trainable_variable:0' shape=() dtype=float32, numpy=3.1467>)
Trainable variables: (<tf.Variable 'trainable_variable:0' shape=() dtype=float32, numpy=3.1467>,)
Non trainable variables: <tf.Variable 'non_trainable_variable:0' shape=() dtype=float32, numpy=12.8>


In [11]:
#Tensorflow linear dense layer
class tfLinear(tf.Module):
  def __init__(self, a, b, name = None):
    super(tfLinear, self).__init__(name = name)
    self.w = tf.Variable(tf.random.normal([a, b]), name = 'weight')
    self.b = tf.Variable(tf.zeros(b,),name = 'bias')
  def __call__(self, inputs):
    out = tf.matmul(inputs, self.w) + self.b
    return tf.nn.relu(out)

mylinear = tfLinear(4,4)
x = tf.ones(shape = [2,4])
mylinear(x)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[2.918643, 0.      , 0.      , 0.      ],
       [2.918643, 0.      , 0.      , 0.      ]], dtype=float32)>

In [17]:
#A complete two layers sequential model
class CModel(tf.Module):
  def __init__(self, name = None):
    super(CModel, self).__init__(name = name)
    self.l1 = tf.keras.layers.Dense(units = 32,activation = 'relu')
    self.l2 = tf.keras.layers.Dense(units = 10, activation = 'softmax')
  def __call__(self, inputs):
    inputs = self.l1(inputs)
    inputs = self.l2(inputs)
    return inputs

cmodel = CModel(name = 'test')
x = tf.ones(shape = (2,2))
cmodel(x)


<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[0.18298353, 0.10244988, 0.06593654, 0.07529923, 0.09981593,
        0.10032313, 0.0728466 , 0.09589595, 0.08598776, 0.11846139],
       [0.18298353, 0.10244988, 0.06593654, 0.07529923, 0.09981593,
        0.10032313, 0.0728466 , 0.09589595, 0.08598776, 0.11846139]],
      dtype=float32)>

In [18]:
print(f"sub_modules: {cmodel.submodules}")

sub_modules: (<tensorflow.python.keras.layers.core.Dense object at 0x7f9907546160>, <tensorflow.python.keras.layers.core.Dense object at 0x7f9906dfc0f0>)


In [19]:
for k in cmodel.variables: #Print detail for each trainable variable
  print(k,'\n')

<tf.Variable 'dense_6/kernel:0' shape=(2, 32) dtype=float32, numpy=
array([[-0.38307554,  0.18238357,  0.25887898,  0.07245845, -0.1901091 ,
         0.41114643,  0.30987307,  0.18072388,  0.1292989 ,  0.345185  ,
        -0.20336202, -0.22214454,  0.38808712, -0.17267317,  0.29445747,
        -0.38313264,  0.15546867, -0.09310195,  0.3474876 , -0.01796907,
         0.06588152,  0.28276607,  0.27711245,  0.05067405,  0.38696352,
         0.29775217, -0.40599662,  0.23794594, -0.21503389, -0.22143203,
         0.25208953,  0.15029296],
       [-0.0788587 ,  0.40235552, -0.08879897,  0.13614234, -0.09167454,
         0.26369378, -0.2512256 , -0.18345875, -0.11385334,  0.14941588,
         0.2637755 ,  0.05049658,  0.12245014, -0.20564067,  0.01229644,
        -0.4021427 , -0.04697248,  0.08644852, -0.13213319, -0.37348834,
        -0.24059285,  0.01186916, -0.2183887 , -0.23583254, -0.32253975,
         0.2607861 , -0.05810753,  0.02656052,  0.00277743,  0.30351993,
        -0.28199285, 

In [42]:
#Differing the input shape to be defined later in the process
class LayerDiff(tf.Module):
  def __init__(self,units, name = None):
    super(LayerDiff, self).__init__(name = name)
    self.built = False
    self.units = units
  def __call__(self,inputs):
    if not self.built:
      self.w = tf.Variable(tf.random.normal([inputs.shape[-1], self.units]), name = 'w')
      self.b = tf.Variable(tf.zeros([self.units]), name = 'b')
      self.built = True
    out = tf.matmul(inputs, self.w) + self.b
    return tf.nn.relu(out)


In [43]:
layer2 = LayerDiff(units = 32, name = 'dense_1')
layer2(tf.constant([[2.0,1.2,4.2,1.3]], dtype = tf.float32))

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[ 2.7706428 ,  5.389567  ,  0.6614248 ,  0.        ,  0.        ,
         4.8701158 ,  6.132825  ,  0.92885435,  0.        , 10.943266  ,
         0.        ,  0.        ,  1.5967556 , 12.485045  ,  1.5606287 ,
         7.0374293 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.2865184 ,  0.        ,  2.7233315 ,  5.779727  ,  4.503691  ,
         0.        ,  0.35464546,  1.0434358 ,  0.        ,  0.        ,
         7.648142  ,  0.        ]], dtype=float32)>

In [46]:
#We can use the above layer by stacking to create a keras sequential model as follow:
class MySequential(tf.Module):
  def __init__(self, name = None):
    super(MySequential, self).__init__(name = name)
    self.l1 = LayerDiff(units = 128, name = 'dense_1')
    self.l2 = LayerDiff(units = 64, name = 'dense_2')

  def __call__(self, inputs):
    inputs = self.l1(inputs)
    return self.l2(inputs)

myseq = MySequential()
print(myseq(tf.constant([[22.1,21.9,11.7,21.8,22.,18.9]])))


tf.Tensor(
[[   0.          0.          0.          0.          0.          0.
   717.5403      0.         50.61929     0.          0.          0.
   651.4185    262.1014      0.          0.        473.8272      0.
     0.        380.11584     0.        623.20984   241.90906     0.
   277.28412   403.50995   352.8406      0.        217.40596     0.
     0.          0.          0.          0.          0.          0.
     0.          0.        562.594       0.        384.679      68.72844
  1078.0778      0.        157.90181   476.36325    26.828857  461.30505
     0.        158.47522     0.          0.          0.          0.
     0.         66.13602     0.          0.        491.17984    56.22271
     0.        470.70755     0.          0.      ]], shape=(1, 64), dtype=float32)
